Data
====

The `geoh5` format allows to store data (values) on different part of an ``Object``.
This section describes the different types of ``Data`` currently supported by `geoh5py`.

Float
-------

The ``Points`` object consists of a list of `vertices` that define the location of ponctual data in 3D space. As for all other `Objects`, it can be created from an array of 3D coordinates and added to any group as follow:

In [ ]:
from geoh5py.workspace import Workspace
from geoh5py.objects import Points
import numpy as np

# Create a blank project
workspace = Workspace("my_project.geoh5")

# Generate a numpy array of xyz locations
n = 100
radius, theta = np.arange(n), np.linspace(0, np.pi*8, n)

x, y = radius * np.cos(theta), radius * np.sin(theta)
z = (x**2. + y**2.)**0.5
xyz = np.c_[x.ravel(), y.ravel(), z.ravel()] # Form a 2D array

# Create the Point object
points = Points.create(
    workspace,       # The target Workspace
    vertices=xyz     # Set vertices
)